<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/16kmodel_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install langchain==0.0.205
!pip install tiktoken==0.3.3
!pip install openai==0.27.2
!pip install pypdf==3.10.0

In [ ]:
import os
import getpass
import tiktoken
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# APIKEY取得
APIKEY = getpass.getpass("APIKEY: ")
os.environ["OPENAI_API_KEY"] = APIKEY

APIKEY: ··········


In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
human_template = """
    下記の#で区切られた文章を英語に翻訳してください

    ####
    {text}
    ####
    """

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
prompt = chat_prompt.format_prompt(text="こんにちは").to_messages()

In [ ]:
response = llm(prompt)

Hello.

In [ ]:
# tokenの数を数えることができる
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
encoding.encode("ねこかいたい")

[2243, 255, 22957, 32149, 16995, 28713, 16995]

In [ ]:
human_template = """
    下記の#で区切られた単語を反対から出力してください

    ####
    {text}
    ####
    """

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
prompt = chat_prompt.format_prompt(text="ねこかいたい").to_messages()

In [ ]:
response = llm(prompt)

いたいかこね

# 要約処理の実装

In [ ]:
def count_token(text):
    print(f"Total token: {len(encoding.encode(text, disallowed_special=()))}")


def preprocess_text(loader):
    docs = loader.load_and_split()
    cnt_token = 0
    content = ""
    for i in range(len(docs)):
        page_content = docs[i].page_content
        cnt_token += len(encoding.encode(page_content, disallowed_special=()))
        if cnt_token > 13_000:
            break
        content += page_content
    count_token(content)
    return content

In [ ]:
def summarizer(content, use_system_message=True):
    template = "あなたはAI機械学習が専門の研究者です。出力は全てマークダウン形式で、各セクションとそれに紐づく内容を日本語で出力します。"
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template = """
    論文: {text}

    上記の論文において、次のステップでタスクを行ってください
    1. 各セクションのタイトルを抽出する
    2. そのタイトルの要約を明確に意味がわかるように行う。必要に応じて専門知識で補完する。

    """
    if not use_system_message:
        human_template = template + human_template
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    if not use_system_message:
        chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
    else:
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )
    prompt = chat_prompt.format_prompt(text=content).to_messages()
    llm = ChatOpenAI(
        temperature=0,
        model="gpt-3.5-turbo-16k",
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    return llm(prompt)

In [ ]:
loader = PyPDFLoader("lora.pdf")

In [ ]:
content = preprocess_text(loader)

Total token: 12131


In [ ]:
content

'LORA: L OW-RANK ADAPTATION OF LARGE LAN-\nGUAGE MODELS\nEdward Hu\x03Yelong Shen\x03Phillip Wallis Zeyuan Allen-Zhu\nYuanzhi Li Shean Wang Lu Wang Weizhu Chen\nMicrosoft Corporation\nfedwardhu, yeshe, phwallis, zeyuana,\nyuanzhil, swang, luw, wzchen g@microsoft.com\nyuanzhil@andrew.cmu.edu\n(Version 2)\nABSTRACT\nAn important paradigm of natural language processing consists of large-scale pre-\ntraining on general domain data and adaptation to particular tasks or domains. As\nwe pre-train larger models, full ﬁne-tuning, which retrains all model parameters,\nbecomes less feasible. Using GPT-3 175B as an example – deploying indepen-\ndent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively\nexpensive. We propose Low-RankAdaptation, or LoRA, which freezes the pre-\ntrained model weights and injects trainable rank decomposition matrices into each\nlayer of the Transformer architecture, greatly reducing the number of trainable pa-\nrameters for downstream tasks. Comp

# システムを使わない場合

In [ ]:
response = summarizer(content, use_system_message=False)

1. ABSTRACT
- 大規模な事前学習と特定のタスクやドメインへの適応からなる自然言語処理の重要なパラダイム
- GPT-3 175Bのような大規模モデルでは、全パラメータの再学習が困難になる
- LoRAという手法を提案し、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減
- LoRAは、GPT-3 175Bの場合、AdamでトレーニングされたGPT-3 175Bと比較して、トレーニング可能なパラメータの数を10,000倍、GPUメモリの要件を3倍に削減できる
- LoRAは、RoBERTa、DeBERTa、GPT-2、GPT-3のモデル品質において、トレーニング可能なパラメータが少なく、トレーニングスループットが高く、アダプターとは異なり追加の推論レイテンシがないにもかかわらず、同等またはそれ以上の性能を発揮する
- LoRAの効果についての実証的な調査を提供し、LoRAをPyTorchモデルと統合するためのパッケージと、RoBERTa、DeBERTa、GPT-2の実装とモデルチェックポイントを提供する

2. INTRODUCTION
- 自然言語処理の多くのアプリケーションでは、1つの大規模な事前学習言語モデルを複数のタスクに適応させることが求められる
- ファインチューニングは、事前学習モデルのすべてのパラメータを再学習するため、大規模なモデルではコストが高くなる
- LoRAは、トレーニング可能なパラメータの数を大幅に削減するための手法であり、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入する
- LoRAは、トレーニング効率を向上させ、ハードウェアの制約を軽減する
- LoRAは、アダプターやプレフィックスチューニングとは異なり、追加の推論レイテンシがない
- LoRAは、他の手法と組み合わせることができる

3. PROBLEM STATEMENT
- ファインチューニングは、事前学習モデルのすべてのパラメータを再学習するため、大規模なモデルではコストが高くなる
- LoRAは、トレーニング可能なパラメータの数を大幅に削減するための手法であり、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入する
-

In [ ]:
from IPython.display import Markdown

Markdown(f'''
{response.content}
''')


1. ABSTRACT
- 大規模な事前学習と特定のタスクやドメインへの適応からなる自然言語処理の重要なパラダイム
- GPT-3 175Bのような大規模モデルでは、全パラメータの再学習が困難になる
- LoRAという手法を提案し、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減
- LoRAは、GPT-3 175Bの場合、AdamでトレーニングされたGPT-3 175Bと比較して、トレーニング可能なパラメータの数を10,000倍、GPUメモリの要件を3倍に削減できる
- LoRAは、RoBERTa、DeBERTa、GPT-2、GPT-3のモデル品質において、トレーニング可能なパラメータが少なく、トレーニングスループットが高く、アダプターとは異なり追加の推論レイテンシがないにもかかわらず、同等またはそれ以上の性能を発揮する
- LoRAの効果についての実証的な調査を提供し、LoRAをPyTorchモデルと統合するためのパッケージと、RoBERTa、DeBERTa、GPT-2の実装とモデルチェックポイントを提供する

2. INTRODUCTION
- 自然言語処理の多くのアプリケーションでは、1つの大規模な事前学習言語モデルを複数のタスクに適応させることが求められる
- ファインチューニングは、事前学習モデルのすべてのパラメータを再学習するため、大規模なモデルではコストが高くなる
- LoRAは、トレーニング可能なパラメータの数を大幅に削減するための手法であり、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入する
- LoRAは、トレーニング効率を向上させ、ハードウェアの制約を軽減する
- LoRAは、アダプターやプレフィックスチューニングとは異なり、追加の推論レイテンシがない
- LoRAは、他の手法と組み合わせることができる

3. PROBLEM STATEMENT
- ファインチューニングは、事前学習モデルのすべてのパラメータを再学習するため、大規模なモデルではコストが高くなる
- LoRAは、トレーニング可能なパラメータの数を大幅に削減するための手法であり、トランスフォーマーアーキテクチャの各層にトレーニング可能なランク分解行列を注入する
- ランクの低いアダプテーション行列を使用することで、ストレージと計算の効率を向上させる
- LoRAは、ファインチューニングと比較して、トレーニングスループットが高く、トレーニング可能なパラメータが少なく、追加の推論レイテンシがないにもかかわらず、モデルの品質が同等またはそれ以上であることを示す
- LoRAは、PyTorchモデルと統合するためのパッケージと、RoBERTa、DeBERTa、GPT-2の実装とモデルチェックポイントを提供する

4. OUR METHOD
- LoRAは、トレーニング可能なランク分解行列を既存の重み行列に注入することで、トレーニング可能なパラメータの数を削減する手法
- LoRAは、トランスフォーマーアーキテクチャの特定の重み行列にのみ適用する
- LoRAは、トレーニング可能なパラメータの数を削減することで、メモリとストレージの使用量を削減し、タスクの切り替えを効率化する
- LoRAは、トレーニング効率を向上させ、ハードウェアの制約を軽減する
- LoRAは、推論時に追加のレイテンシを導入せず、他の手法と組み合わせることができる

5. EMPIRICAL EXPERIMENTS
- RoBERTa、DeBERTa、GPT-2、GPT-3のモデルにLoRAを適用し、GLUEベンチマークや他のタスクでのパフォーマンスを評価する
- LoRAは、他の手法と比較して優れたパフォーマンスを発揮することを示す
- LoRAは、トレーニング可能なパラメータの数を削減し、トレーニング効率を向上させる
- LoRAは、ファインチューニングと比較して追加の推論レイテンシがない
- LoRAは、PyTorchモデルと統合するためのパッケージと、RoBERTa、DeBERTa、GPT-2の実装とモデルチェックポイントを提供する

6. RELATED WORKS
- トランスフォーマーモデルは、自然言語処理の分野で広く使用されている
- ファインチューニングやプレフィックスチューニングなど、パラメータ効率の改善手法が提案されている
- 低ランク構造は、機械学習の分野で一般的に使用されている
- LoRAは、他の手法と比較して優れたパフォーマンスを発揮することを示す
- LoRAは、他の手法と組み合わせることができる


In [ ]:
count_token(response.content)

Total token: 1812


# システムを使う場合

In [ ]:
response = summarizer(content, use_system_message=True)

1. ABSTRACT
- 論文の要点: LoRAは、大規模な言語モデルを特定のタスクやドメインに適応させるための低ランク適応手法です。LoRAは、事前学習済みモデルの重みを凍結し、各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減します。LoRAは、GPT-3 175Bと比較して、トレーニング可能なパラメータの数を10,000倍、GPUメモリの要件を3倍に削減することができます。また、LoRAは、RoBERTa、DeBERTa、GPT-2、GPT-3のモデル品質において、トレーニング可能なパラメータが少なく、トレーニングスループットが高く、アダプターとは異なり追加の推論レイテンシがないにもかかわらず、同等以上のパフォーマンスを発揮します。

2. INTRODUCTION
- 論文の背景: 自然言語処理の重要なパラダイムは、一般的なドメインのデータでの大規模な事前学習と、特定のタスクやドメインへの適応です。しかし、より大きなモデルを事前学習するにつれて、全パラメータを再学習する完全なファインチューニングは実現困難になります。そこで、LoRAでは、事前学習済みモデルの重みを凍結し、各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減します。

3. PROBLEM STATEMENT
- 問題の概要: ファインチューニングは、事前学習済みモデルのすべてのパラメータを更新するため、パラメータ数が元のモデルと同じだけ必要になります。しかし、大規模なモデルでは、ファインチューニングはモデルのデプロイメントにおける重要な課題となります。LoRAは、ファインチューニングと比較してトレーニング可能なパラメータの数を大幅に削減し、モデルの品質を維持することができます。

4. OUR METHOD
- メソッドの概要: LoRAは、事前学習済みモデルの重みにトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を削減します。LoRAは、モデルのトレーニング効率を向上させ、ハードウェアの制約を緩和します。また、LoRAは、ファインチューニングと比較して推論レイテンシを追加しないため、デプロイメント時の効率も向上させます。

5. EMPIRICAL EXPE

In [ ]:
Markdown(f'''
{response.content}
''')


1. ABSTRACT
- 論文の要点: LoRAは、大規模な言語モデルを特定のタスクやドメインに適応させるための低ランク適応手法です。LoRAは、事前学習済みモデルの重みを凍結し、各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減します。LoRAは、GPT-3 175Bと比較して、トレーニング可能なパラメータの数を10,000倍、GPUメモリの要件を3倍に削減することができます。また、LoRAは、RoBERTa、DeBERTa、GPT-2、GPT-3のモデル品質において、トレーニング可能なパラメータが少なく、トレーニングスループットが高く、アダプターとは異なり追加の推論レイテンシがないにもかかわらず、同等以上のパフォーマンスを発揮します。

2. INTRODUCTION
- 論文の背景: 自然言語処理の重要なパラダイムは、一般的なドメインのデータでの大規模な事前学習と、特定のタスクやドメインへの適応です。しかし、より大きなモデルを事前学習するにつれて、全パラメータを再学習する完全なファインチューニングは実現困難になります。そこで、LoRAでは、事前学習済みモデルの重みを凍結し、各層にトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を大幅に削減します。

3. PROBLEM STATEMENT
- 問題の概要: ファインチューニングは、事前学習済みモデルのすべてのパラメータを更新するため、パラメータ数が元のモデルと同じだけ必要になります。しかし、大規模なモデルでは、ファインチューニングはモデルのデプロイメントにおける重要な課題となります。LoRAは、ファインチューニングと比較してトレーニング可能なパラメータの数を大幅に削減し、モデルの品質を維持することができます。

4. OUR METHOD
- メソッドの概要: LoRAは、事前学習済みモデルの重みにトレーニング可能なランク分解行列を注入することで、トレーニング可能なパラメータの数を削減します。LoRAは、モデルのトレーニング効率を向上させ、ハードウェアの制約を緩和します。また、LoRAは、ファインチューニングと比較して推論レイテンシを追加しないため、デプロイメント時の効率も向上させます。

5. EMPIRICAL EXPERIMENTS
- 実験結果の概要: LoRAは、RoBERTa、DeBERTa、GPT-2、GPT-3のモデルにおいて、トレーニング可能なパラメータの数を削減しながら、モデルの品質を維持することができます。また、GPT-3 175Bにおいても、LoRAはファインチューニングと比較して優れたパフォーマンスを発揮します。

6. RELATED WORKS
- 関連研究の概要: LoRAは、他のパラメータ効率の高い適応手法と比較しても優れたパフォーマンスを発揮します。また、LoRAは、事前学習済み言語モデルを下流のタスクに適応させるための基本原則を明らかにするものです。


In [ ]:
human_template="""
次の論文から下記の質問に答えよ
論文:{thesis}

質問:{question}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
prompt = chat_prompt.format_prompt(thesis=content, question="どのような評価を行って、モデルの品質が維持できることを確かめたのですか？").to_messages()
response = llm(prompt)

論文では、LoRAの性能を評価するために、RoBERTa、DeBERTa、GPT-2、およびGPT-3のさまざまなタスクでの実験を行いました。具体的には、GLUEベンチマーク、E2E NLG Challenge、WikiSQL、MultiNLI、SAMSumなどのタスクでの性能を評価しました。これらの実験により、LoRAがフルファインチューニングと同等またはそれ以上の性能を提供することが示されました。また、LoRAがパラメータ数を大幅に削減できることも確認されました。

In [ ]:
Markdown(f'''
{response.content}
''')


論文では、LoRAの性能を評価するために、RoBERTa、DeBERTa、GPT-2、およびGPT-3のさまざまなタスクでの実験を行いました。具体的には、GLUEベンチマーク、E2E NLG Challenge、WikiSQL、MultiNLI、SAMSumなどのタスクでの性能を評価しました。これらの実験により、LoRAがフルファインチューニングと同等またはそれ以上の性能を提供することが示されました。また、LoRAがパラメータ数を大幅に削減できることも確認されました。


In [ ]:
loader = PyPDFLoader("less_is_more.pdf")
content = preprocess_text(loader)

In [ ]:
# システムを使わない場合
response = summarizer(content, use_system_message=False)

In [ ]:
# システムを使う場合
response = summarizer(content, use_system_message=True)

# 質問もできる

In [ ]:
human_template="""
次の論文から下記の質問に答えよ
論文:{thesis}

質問:{question}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
prompt = chat_prompt.format_prompt(thesis=content, question="約1,000のプロンプトとは具体的にどういうものですか？").to_messages()
response = llm(prompt)